In [713]:
# from pyspark import SparkContext 
# from pyspark.sql import SQLContext 
import pandas as pd
import numpy as np
# from pandas import DataFrame
# sc = SparkContext.getOrCreate()
# sqlc=SQLContext(sc)

import math
import matplotlib.pyplot as plt
from scipy.stats import norm, entropy 
from scipy.stats import probplot
import matplotlib.pyplot as plt
from scipy.stats import logistic
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.utils import resample
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from scipy.spatial.distance import cosine, correlation
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

In [791]:
#reading csv file data in a dataframe
#[1]https://stackoverflow.com/questions/46311133/creating-dynamic-data-frames-in-python?rq=1
# df_names=['df07','df08','df09','df10','df11','df12','df13','df14','df15','df16','df17','df18']
# years=['2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018']
df_names=['df14','df15','df16','df17','df18']
years=['2014','2015','2016','2017','2018']
d = {}
for name,i in zip(df_names,years):
    d[name] = pd.DataFrame()
    d[name]=pd.read_csv('data-csv/chs'+str(i)+'_public.csv') 

# Collecting data from all years and renamimng columns

In [792]:
#[1]https://www.geeksforgeeks.org/how-to-rename-columns-in-pandas-dataframe/
df18=d['df18'][["cid","exercise18","cyclingfreq","diabetes18",
               "toldhighbp18","nutrition46","nutrition47","ssb",
              "generalhealth","maritalstatus18",
                "daysalc30","fruitveg18","mood55",
                "everhivtest18","hepbvaccine18",
                "imputed_sexuallyactive18",'imputed_sexpartner']]

df18=df18.rename(columns={"exercise18":"exercise","toldhighbp18":"toldhighbp","diabetes18":"diabetes",
                          "maritalstatus18":"maritalstatus","fruitveg18":"fruitveg","mood55":"mood",
                         "everhivtest18":"everhivtest","hepbvaccine18":"vaccine",
                        "imputed_sexuallyactive18":"sexuallyactive","imputed_sexpartner":"sexpartner"})

df17=d['df17'][["cid","exercise17","cyclingfreq","nutrition46","nutrition47","ssb",
              "diabetes17","toldhighbp17","generalhealth","maritalstatus17",
               "daysalc30","fruitveg17","mood55",
               "everhivtest17","fluvaccineshot",
                "sexuallyactive17",'imputed_sexpartner']]

df17=df17.rename(columns={"exercise17":"exercise","toldhighbp17":"toldhighbp","diabetes17":"diabetes",
                          "maritalstatus17":"maritalstatus","fruitveg17":"fruitveg","mood55":"mood",
                         "everhivtest17":"everhivtest","fluvaccineshot":"vaccine",
                        "sexuallyactive17":"sexuallyactive","imputed_sexpartner":"sexpartner"})

df16=d['df16'][["cid","exercise16","ssb",
              "diabetes16","toldhighbp16","generalhealth","maritalstatus16",
               "daysalc30","fruitveg","mood63",
               "everhivtest16","fluvaccineshot",
                "sexuallyactive16",'sexpartner']]

df16=df16.rename(columns={"exercise16":"exercise","toldhighbp16":"toldhighbp","diabetes16":"diabetes",
                          "maritalstatus16":"maritalstatus","mood63":"mood",
                         "everhivtest16":"everhivtest","fluvaccineshot":"vaccine",
                        "sexuallyactive16":"sexuallyactive"})

df15=d['df15'][["cid","exercise15","ssb",
              "diabetes15","toldhighbp15","generalhealth","maritalstatus15",
               "daysalc30","fruitveg","mood1",
               "everhivtest15","fluvaccineshot",
                "sexuallyactive15",'sexpartner']]

df15=df15.rename(columns={"exercise15":"exercise","toldhighbp15":"toldhighbp",
                          "diabetes15":"diabetes","maritalstatus15":"maritalstatus","mood1":"mood",
                         "everhivtest15":"everhivtest","fluvaccineshot":"vaccine",
                        "sexuallyactive15":"sexuallyactive"})

df14=d['df14'][["cid","exercise14","ssb",
              "diabetes14","toldhighbp14","generalhealth","maritalstatus14","tolddepression",
               "everhivtest14","fluvaccineshot",
                "sexuallyactive14",'sexpartner']]

df14=df14.rename(columns={"exercise14":"exercise","toldhighbp14":"toldhighbp",
                          "diabetes14":"diabetes","maritalstatus14":"maritalstatus","tolddepression":"mood",
                         "everhivtest14":"everhivtest","fluvaccineshot":"vaccine",
                        "sexuallyactive14":"sexuallyactive"})

df18.head()

,cid,exercise,cyclingfreq,diabetes,toldhighbp,nutrition46,nutrition47,ssb,generalhealth,maritalstatus,daysalc30,fruitveg,mood,everhivtest,vaccine,sexuallyactive,sexpartner
0,1900001,1.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,5.0,7.0,2.0,2.0,2.0,2.0,1.0,4.0
1,1900002,1.0,5.0,2.0,2.0,1.0,1.0,1.0,2.0,5.0,0.0,2.0,1.0,1.0,1.0,1.0,2.0
2,1900003,1.0,5.0,2.0,2.0,1.0,2.0,2.0,4.0,NaN,12.9,2.0,3.0,1.0,2.0,1.0,2.0
3,1900004,1.0,5.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,4.0,2.0,2.0,1.0,1.0,2.0,1.0
4,1900005,2.0,2.0,2.0,2.0,0.0,2.0,2.0,3.0,5.0,0.0,2.0,2.0,2.0,1.0,2.0,1.0


# Combining data from all years

In [694]:
#[1]https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html
df=df18.append(df17,ignore_index=True)
df=df.append(df16,ignore_index=True)
df=df.append(df15,ignore_index=True)
df=df.append(df14,ignore_index=True)
df

,cid,cyclingfreq,daysalc30,diabetes,everhivtest,exercise,fruitveg,generalhealth,maritalstatus,mood,nutrition46,nutrition47,sexpartner,sexuallyactive,ssb,toldhighbp,vaccine
0,1900001,2.0,7.0,2.0,2.0,1.0,2.0,2.0,5.0,2.0,2.0,1.0,4.0,1.0,2.0,1.0,2.0
1,1900002,5.0,0.0,2.0,1.0,1.0,2.0,2.0,5.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0
2,1900003,5.0,12.9,2.0,1.0,1.0,2.0,4.0,NaN,3.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0
3,1900004,5.0,4.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0
4,1900005,2.0,0.0,2.0,2.0,2.0,2.0,3.0,5.0,2.0,0.0,2.0,1.0,2.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48810,1514236,NaN,NaN,2.0,2.0,2.0,NaN,4.0,5.0,1.0,NaN,NaN,1.0,2.0,2.0,1.0,2.0
48811,1514237,NaN,NaN,2.0,1.0,1.0,NaN,2.0,2.0,1.0,NaN,NaN,2.0,1.0,1.0,2.0,2.0
48812,1514238,NaN,NaN,2.0,2.0,1.0,NaN,2.0,3.0,2.0,NaN,NaN,1.0,2.0,2.0,1.0,1.0
48813,1514239,NaN,NaN,2.0,1.0,1.0,NaN,3.0,5.0,1.0,NaN,NaN,1.0,2.0,2.0,2.0,2.0


In [695]:
df.isnull().sum()

cid                   0
cyclingfreq       28781
daysalc30          8922
diabetes            125
everhivtest        1577
exercise             71
fruitveg           9692
generalhealth       299
maritalstatus       436
mood                555
nutrition46       29054
nutrition47       29138
sexpartner         2802
sexuallyactive     3622
ssb                 519
toldhighbp          174
vaccine             239
dtype: int64

In [696]:
#[1]https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html
#Dataframe after removing columns with many Null values
df=df[["cid","exercise","diabetes",
               "toldhighbp","ssb",
              "generalhealth","maritalstatus","daysalc30","mood","fruitveg",
              "everhivtest","vaccine",
                "sexuallyactive",'sexpartner']]
df

,cid,exercise,diabetes,toldhighbp,ssb,generalhealth,maritalstatus,daysalc30,mood,fruitveg,everhivtest,vaccine,sexuallyactive,sexpartner
0,1900001,1.0,2.0,1.0,2.0,2.0,5.0,7.0,2.0,2.0,2.0,2.0,1.0,4.0
1,1900002,1.0,2.0,2.0,1.0,2.0,5.0,0.0,1.0,2.0,1.0,1.0,1.0,2.0
2,1900003,1.0,2.0,2.0,2.0,4.0,NaN,12.9,3.0,2.0,1.0,2.0,1.0,2.0
3,1900004,1.0,2.0,2.0,2.0,2.0,2.0,4.0,2.0,2.0,1.0,1.0,2.0,1.0
4,1900005,2.0,2.0,2.0,2.0,3.0,5.0,0.0,2.0,2.0,2.0,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48810,1514236,2.0,2.0,1.0,2.0,4.0,5.0,NaN,1.0,NaN,2.0,2.0,2.0,1.0
48811,1514237,1.0,2.0,2.0,1.0,2.0,2.0,NaN,1.0,NaN,1.0,2.0,1.0,2.0
48812,1514238,1.0,2.0,1.0,2.0,2.0,3.0,NaN,2.0,NaN,2.0,1.0,2.0,1.0
48813,1514239,1.0,2.0,2.0,2.0,3.0,5.0,NaN,1.0,NaN,1.0,2.0,2.0,1.0


In [697]:

df.daysalc30=df.daysalc30.fillna(value=int(df.daysalc30.mode()))
df.fruitveg=df.fruitveg.fillna(value=int(df.fruitveg.mode()))
df.everhivtest=df.everhivtest.fillna(value=int(df.everhivtest.mode()))
df.sexuallyactive=df.sexuallyactive.fillna(value=int(df.sexuallyactive.mode()))
df.sexpartner=df.sexpartner.fillna(value=int(df.sexpartner.mode()))
# df=df.fillna(value=df.mode())
# fill_value


In [700]:
df.isnull().sum()

cid               0
exercise          0
diabetes          0
toldhighbp        0
ssb               0
generalhealth     0
maritalstatus     0
daysalc30         0
mood              0
fruitveg          0
everhivtest       0
vaccine           0
sexuallyactive    0
sexpartner        0
dtype: int64

In [699]:
df=df.dropna()

In [553]:
corr = df.corr()
corr

,cid,exercise,diabetes,toldhighbp,ssb,generalhealth,maritalstatus,daysalc30,mood,fruitveg
cid,1.000000,0.013985,-0.003805,0.006161,-0.004296,-0.003356,0.008159,-0.000881,-0.552364,0.026937
exercise,0.013985,1.000000,-0.080643,-0.074706,-0.073992,0.206451,-0.021481,-0.084824,0.029950,-0.148788
diabetes,-0.003805,-0.080643,1.000000,0.332131,-0.049484,-0.305361,0.046879,0.132583,-0.028998,0.047293
toldhighbp,0.006161,-0.074706,0.332131,1.000000,-0.011728,-0.322455,0.063865,0.070450,-0.035241,0.046750
ssb,-0.004296,-0.073992,-0.049484,-0.011728,1.000000,-0.038839,-0.118001,0.031109,-0.019279,0.137016
generalhealth,-0.003356,0.206451,-0.305361,-0.322455,-0.038839,1.000000,-0.017444,-0.177398,0.096864,-0.147431
maritalstatus,0.008159,-0.021481,0.046879,0.063865,-0.118001,-0.017444,1.000000,-0.014932,0.019422,-0.081538
daysalc30,-0.000881,-0.084824,0.132583,0.070450,0.031109,-0.177398,-0.014932,1.000000,-0.004271,0.058884
mood,-0.552364,0.029950,-0.028998,-0.035241,-0.019279,0.096864,0.019422,-0.004271,1.000000,-0.028951
fruitveg,0.026937,-0.148788,0.047293,0.046750,0.137016,-0.147431,-0.081538,0.058884,-0.028951,1.000000


In [554]:
#[1]https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html
#[2]https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
#[3]https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60
#[4]https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
#[5]https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error
from math import sqrt

#function for rmse calculation of model
def model_apply(x,y,algo,n):
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,random_state=0)
    scaler = StandardScaler()
    # Fit on training set only.
    scaler.fit(x_train)
    # Apply transform to both the training set and the test set.
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    
    pca = PCA(n_components=n,tol=.95)
    pca.fit(x_test)
    print(pca.explained_variance_ratio_) 
    
    x_train = pca.transform(x_train)
    x_test = pca.transform(x_test)
    
#     principalDf = pd.DataFrame(data = x_train
#              , columns = ['pc1', 'pc2','pc3'])
#     print(principalDf)

    #model selection
    if algo==1:
        model=LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial',max_iter=200)
    elif algo==2:
        model=GaussianNB()
    elif algo==3:
        model=DecisionTreeClassifier()
    elif algo==4:
        model=svm.SVC(gamma='scale')
    elif algo==5:
        model=Perceptron(tol=1e-3, random_state=0)
    elif algo==6:
        model=RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
        
        # Train the model using the training sets
    model.fit(x_train,y_train.values.ravel())

    # Predict Output
    y_pred = model.predict(x_test)
    print ("Predicted Value:", y_pred)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    return(rmse)

# Predicting General health

In [569]:
#preparing training and test data
x=df.drop(columns=["cid","generalhealth"]) #source feature
y=df[["generalhealth"]] #target feature
model_apply(x,y,1,6)

[0.18478733 0.1620335  0.12595995 0.12557436 0.11441092 0.1044053 ]
Predicted Value: [3. 4. 3. ... 3. 3. 3.]
Accuracy: 0.3686458778299872
[[   0  909  533   19    1]
 [   0 1333 1055   98    4]
 [   0 1028 1666  332    8]
 [   0  335  960  434   21]
 [   0   67  279  263   19]]
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00      1462
         2.0       0.36      0.54      0.43      2490
         3.0       0.37      0.55      0.44      3034
         4.0       0.38      0.25      0.30      1750
         5.0       0.36      0.03      0.06       628

    accuracy                           0.37      9364
   macro avg       0.29      0.27      0.25      9364
weighted avg       0.31      0.37      0.32      9364



/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


1.0541713484781101

In [571]:
#preparing training and test data
x=df[["exercise"]] #source feature
y=df[["generalhealth"]] #target feature
model_apply(x,y,3,1)

[1.]
Predicted Value: [3. 3. 3. ... 3. 3. 3.]
Accuracy: 0.3240068346860316
[[   0    0 1462    0    0]
 [   0    0 2490    0    0]
 [   0    0 3034    0    0]
 [   0    0 1750    0    0]
 [   0    0  628    0    0]]
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00      1462
         2.0       0.00      0.00      0.00      2490
         3.0       0.32      1.00      0.49      3034
         4.0       0.00      0.00      0.00      1750
         5.0       0.00      0.00      0.00       628

    accuracy                           0.32      9364
   macro avg       0.06      0.20      0.10      9364
weighted avg       0.10      0.32      0.16      9364



/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


1.1599908674094386

In [573]:
#preparing training and test data
x=df[["diabetes"]] #source feature
y=df[["generalhealth"]] #target feature
model_apply(x,y,1,1)

[1.]
Predicted Value: [3. 3. 3. ... 4. 3. 3.]
Accuracy: 0.33351131994873984
[[   0    0 1418   44    0]
 [   0    0 2358  132    0]
 [   0    0 2629  405    0]
 [   0    0 1256  494    0]
 [   0    0  347  281    0]]
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00      1462
         2.0       0.00      0.00      0.00      2490
         3.0       0.33      0.87      0.48      3034
         4.0       0.36      0.28      0.32      1750
         5.0       0.00      0.00      0.00       628

    accuracy                           0.33      9364
   macro avg       0.14      0.23      0.16      9364
weighted avg       0.17      0.33      0.21      9364



/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


1.1453525877179591

In [574]:
#preparing training and test data
x=df[["toldhighbp"]] #source feature
y=df[["generalhealth"]] #target feature
model_apply(x,y,3,1)

[1.]
Predicted Value: [2. 3. 2. ... 2. 2. 3.]
Accuracy: 0.3172789406236651
[[   0 1271  191    0    0]
 [   0 1901  589    0    0]
 [   0 1964 1070    0    0]
 [   0  807  943    0    0]
 [   0  204  424    0    0]]
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00      1462
         2.0       0.31      0.76      0.44      2490
         3.0       0.33      0.35      0.34      3034
         4.0       0.00      0.00      0.00      1750
         5.0       0.00      0.00      0.00       628

    accuracy                           0.32      9364
   macro avg       0.13      0.22      0.16      9364
weighted avg       0.19      0.32      0.23      9364



/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


1.1456788790830235

In [575]:
#preparing training and test data
x=df[["ssb"]] #source feature
y=df[["generalhealth"]] #target feature
model_apply(x,y,3,1)

[1.]
Predicted Value: [3. 3. 3. ... 3. 3. 3.]
Accuracy: 0.3240068346860316
[[   0    0 1462    0    0]
 [   0    0 2490    0    0]
 [   0    0 3034    0    0]
 [   0    0 1750    0    0]
 [   0    0  628    0    0]]
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00      1462
         2.0       0.00      0.00      0.00      2490
         3.0       0.32      1.00      0.49      3034
         4.0       0.00      0.00      0.00      1750
         5.0       0.00      0.00      0.00       628

    accuracy                           0.32      9364
   macro avg       0.06      0.20      0.10      9364
weighted avg       0.10      0.32      0.16      9364



/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


1.1599908674094386

# Predicting Cardiovascular health

In [337]:
#preparing training and test data
x=df.drop(columns=["cid","generalhealth","toldhighbp"]) #source feature
y=df[["toldhighbp"]] #target feature
model_apply(x,y,1,3)

[0.28985337 0.26970711 0.23198711]
Predicted Value: [2. 2. 1. ... 2. 2. 2.]
Accuracy: 0.7232225885535548
[[1227 2810]
 [ 464 7328]]
              precision    recall  f1-score   support

         1.0       0.73      0.30      0.43      4037
         2.0       0.72      0.94      0.82      7792

    accuracy                           0.72     11829
   macro avg       0.72      0.62      0.62     11829
weighted avg       0.72      0.72      0.68     11829



0.5260963898815931

# Predicting Diabetes

In [338]:
df.diabetes.value_counts()

2.0    40571
1.0     6745
Name: diabetes, dtype: int64

# Up Sampling

In [339]:
from sklearn.utils import resample
# Separate majority and minority classes
df_majority = df[df.diabetes==2]
df_minority = df[df.diabetes==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=40911,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_upsampled.diabetes.value_counts()

1.0    40911
2.0    40571
Name: diabetes, dtype: int64

In [345]:
#preparing training and test data
x=df_upsampled.drop(columns=["cid","generalhealth","diabetes"]) #source feature
y=df_upsampled[["diabetes"]] #target feature
model_apply(x,y,3,1)

[0.28735312]
Predicted Value: [1. 2. 1. ... 2. 1. 1.]
Accuracy: 0.7238721712237985
[[7647 2719]
 [2906 7099]]
              precision    recall  f1-score   support

         1.0       0.72      0.74      0.73     10366
         2.0       0.72      0.71      0.72     10005

    accuracy                           0.72     20371
   macro avg       0.72      0.72      0.72     20371
weighted avg       0.72      0.72      0.72     20371



0.5254786663378461

# Down Sampling

In [346]:
#Down Sampling
df_majority = df[df.diabetes==2]
df_minority = df[df.diabetes==1]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=6806,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.diabetes.value_counts()

2.0    6806
1.0    6745
Name: diabetes, dtype: int64

In [351]:
#preparing training and test data
x=df_downsampled.drop(columns=["cid","generalhealth","diabetes"]) #source feature
y=df_downsampled[["diabetes"]] #target feature
model_apply(x,y,1,3)

[0.29216052 0.26787461 0.2230817 ]
Predicted Value: [1. 2. 1. ... 2. 2. 2.]
Accuracy: 0.7175324675324676
[[1288  462]
 [ 495 1143]]
              precision    recall  f1-score   support

         1.0       0.72      0.74      0.73      1750
         2.0       0.71      0.70      0.70      1638

    accuracy                           0.72      3388
   macro avg       0.72      0.72      0.72      3388
weighted avg       0.72      0.72      0.72      3388



0.5314767468737767

# Marital Status based analysis

In [352]:
df_martial=df[["maritalstatus","toldhighbp","generalhealth"]]
df_martial=df_martial.dropna()

In [354]:
#preparing training and test data
x=df_martial[["maritalstatus"]] #source feature
y=df_martial[["toldhighbp"]] #target feature
model_apply(x,y,3,1)

[1.]
Predicted Value: [2. 2. 2. ... 2. 2. 2.]
Accuracy: 0.6798545946402909
[[ 658 3379]
 [ 408 7384]]
              precision    recall  f1-score   support

         1.0       0.62      0.16      0.26      4037
         2.0       0.69      0.95      0.80      7792

    accuracy                           0.68     11829
   macro avg       0.65      0.56      0.53     11829
weighted avg       0.66      0.68      0.61     11829



0.5658139317476278

In [357]:
#preparing training and test data
x=df_martial[["maritalstatus"]] #source feature
y=df_martial[["generalhealth"]] #target feature
model_apply(x,y,1,1)

[1.]
Predicted Value: [3. 3. 3. ... 3. 3. 3.]
Accuracy: 0.32394961535210076
[[   0    0 1866    0    0]
 [   0    0 3154    0    0]
 [   0    0 3832    0    0]
 [   0    0 2190    0    0]
 [   0    0  787    0    0]]
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00      1866
         2.0       0.00      0.00      0.00      3154
         3.0       0.32      1.00      0.49      3832
         4.0       0.00      0.00      0.00      2190
         5.0       0.00      0.00      0.00       787

    accuracy                           0.32     11829
   macro avg       0.06      0.20      0.10     11829
weighted avg       0.10      0.32      0.16     11829



/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


1.1614165167166444

In [784]:
#function for rmse calculation of model
def model_prediction(x,y,z,algo,n):
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,random_state=0)
    scaler = StandardScaler()
    # Fit on training set only.
    scaler.fit(x_train)
    # Apply transform to both the training set and the test set.
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    
    pca = PCA(n_components=n,tol=.95)
    pca.fit(x_test)
    print("\n",pca.explained_variance_ratio_) 
    
    x_train = pca.transform(x_train)
    x_test = pca.transform(x_test)
    z_test=pca.transform(z)

    #model selection
    if algo==1:
        model=LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial',max_iter=200)
    elif algo==2:
        model=GaussianNB()
    elif algo==3:
        model=DecisionTreeClassifier()
    elif algo==4:
        model=svm.SVC(gamma='scale')
    elif algo==5:
        model=Perceptron(tol=1e-3, random_state=0)
    elif algo==6:
        model=RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
    elif algo==7:
        model = LinearRegression()
        
        # Train the model using the training sets
    model.fit(x_train,y_train.values.ravel())

    # Predict Output
    y_pred = model.predict(x_test)
    print ("Predicted Value:", y_pred)
#     print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
#     print(confusion_matrix(y_test,y_pred))
#     print(classification_report(y_test,y_pred))
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    print(rmse)
    z_pred=model.predict(z_test)
    print("General Health rating predicted is :",z_pred)
    return(z_pred)

# Demonstration

In [785]:
len(df.columns)

14

In [786]:
values=[]
for i in df.iloc[0]:
    values.append(int(i))
values
#6 is general health

[1900001, 1, 2, 1, 2, 2, 5, 7, 2, 2, 2, 2, 1, 4]

In [787]:
column_names=["cid","exercise","diabetes",
               "toldhighbp","ssb",
              "generalhealth","maritalstatus","daysalc30","fruitveg","mood",
                 "everhivtest","vaccine",
                "sexuallyactive",'sexpartner']

print("Individual Id:",values[0],"has general health:",values[5])
x=df.drop(columns=["cid","generalhealth"]) #source feature
y=df[["generalhealth"]] #target feature
z=pd.DataFrame([values],columns=column_names).drop(columns=["cid","generalhealth"])
z.iloc[0]

Individual Id: 1900001 has general health: 2


exercise          1
diabetes          2
toldhighbp        1
ssb               2
maritalstatus     5
daysalc30         7
fruitveg          2
mood              2
everhivtest       2
vaccine           2
sexuallyactive    1
sexpartner        4
Name: 0, dtype: int64

In [788]:
result=[model_prediction(x,y,z,1,11),model_prediction(x,y,z,7,11), 
        model_prediction(x,y,z,3,11),model_prediction(x,y,z,6,11)]


 [0.17737298 0.10914102 0.10138388 0.0898975  0.08474933 0.07802106
 0.07638185 0.07274515 0.07058367 0.06567311 0.05357634]
Predicted Value: [3. 2. 3. ... 3. 4. 3.]
1.064417517856269
General Health rating predicted is : [2.]

 [0.17737298 0.10914102 0.10138388 0.0898975  0.08474933 0.07802106
 0.07638185 0.07274515 0.07058367 0.06567311 0.05357634]
Predicted Value: [2.96560201 2.30025074 2.96482776 ... 3.2801597  3.70621401 3.00836758]
1.0159031053164238
General Health rating predicted is : [1.08654697]

 [0.17737298 0.10914102 0.10138388 0.0898975  0.08474933 0.07802106
 0.07638185 0.07274515 0.07058367 0.06567311 0.05357634]
Predicted Value: [2. 2. 4. ... 4. 4. 3.]
1.29955571854078
General Health rating predicted is : [1.]

 [0.17737298 0.10914102 0.10138388 0.0898975  0.08474933 0.07802106
 0.07638185 0.07274515 0.07058367 0.06567311 0.05357634]
Predicted Value: [3. 2. 3. ... 3. 3. 3.]
1.1257243992794084
General Health rating predicted is : [3.]
